<a href="https://colab.research.google.com/github/adalbertogc1/AutomaticTaggingSample/blob/main/Auto_tagging_using_Semantic_Similarity_with_TF_Hub_Universal_Encoder.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This notebook illustrates how to access the Universal Sentence Encoder and use it for sentence similarity and sentence classification tasks.

The Universal Sentence Encoder makes getting sentence level embeddings as easy as it has historically been to lookup the embeddings for individual words. The sentence embeddings can then be trivially used to compute sentence level meaning similarity as well as to enable better performance on downstream classification tasks using less supervised training data.


# Getting Started

This section sets up the environment for access to the Universal Sentence Encoder on TF Hub and provides examples of applying the encoder to words, sentences, and paragraphs.

In [ ]:
# Install the latest Tensorflow version.
!pip3 install --quiet "tensorflow>=1.7"
# Install TF-Hub.
!pip3 install --quiet tensorflow-hub
!pip3 install --quiet seaborn

More detailed information about installing Tensorflow can be found at [https://www.tensorflow.org/install/](https://www.tensorflow.org/install/).

In [ ]:
from absl import logging

import tensorflow as tf
import tensorflow_hub as hub
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import re
import seaborn as sns

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [ ]:
path = "https://drive.google.com/open?id=1lp3LqrtaowAH9uIK4WtzUUT6_0dzpduX"
df= pd.read_excel("Dataset.xlsx")

In [ ]:
df.head(14)

,RawName,Tag1,Tag2,Tag3,Tag4,Tag5
0,Control Enable,control,enable,NaN,NaN,NaN
1,Control Enable St,control,enable,status,NaN,NaN
2,Control Enable Cmd,control,enable,command,NaN,NaN
3,IPW4-88 2 Room Temp,temp,room,id,NaN,NaN
4,Room SPA Feedback,room,feedback,NaN,NaN,NaN
5,Valve Position,valve,NaN,NaN,NaN,NaN
6,Room Final SPA,room,NaN,NaN,NaN,NaN
7,Final Room SPA,room,NaN,NaN,NaN,NaN
8,Valve O/ride Demand,override,NaN,NaN,NaN,NaN
9,Valve O/ride Output,output,NaN,NaN,NaN,NaN


In [ ]:
df1=pd.get_dummies(df, columns=["Tag1","Tag2","Tag3","Tag4","Tag5"],sparse=False).groupby(lambda x: x.split('_')[-1], axis=1).sum()
df1.head(14)

KeyError: ignored

In [ ]:
module_url = "https://tfhub.dev/google/universal-sentence-encoder/2" #@param ["https://tfhub.dev/google/universal-sentence-encoder/2", "https://tfhub.dev/google/universal-sentence-encoder-large/3"]

In [ ]:
# Import the Universal Sentence Encoder's TF Hub module
embed = hub.Module(module_url)

# Compute a representation for each message, showing various lengths supported.

messages = ["Control Enable", "Control Enable St", "Control Enable Cmd", "IPW4-88 2 Room Temp", "Room SPA Feedback", "Valve Position", "Room Final SPA", "Final Room SPA", "Valve O/ride Demand", "Valve O/ride Output", "Control Enable", "Control Enable St", "Control Enable Cmd", "IPW4-89 2 Room Temp"]

# Reduce logging output.
logging.set_verbosity(logging.ERROR)

with tf.Session() as session:
  session.run([tf.global_variables_initializer(), tf.tables_initializer()])
  message_embeddings = session.run(embed(messages))

  for i, message_embedding in enumerate(np.array(message_embeddings).tolist()):
    print("Message: {}".format(messages[i]))
    print("Embedding size: {}".format(len(message_embedding)))
    message_embedding_snippet = ", ".join(
        (str(x) for x in message_embedding[:3]))
    print("Embedding: [{}, ...]\n".format(message_embedding_snippet))

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


Message: Control Enable
Embedding size: 512
Embedding: [-0.015263891778886318, -0.07776880264282227, -0.011007686145603657, ...]

Message: Control Enable St
Embedding size: 512
Embedding: [0.025552142411470413, -0.08018751442432404, -0.005377942230552435, ...]

Message: Control Enable Cmd
Embedding size: 512
Embedding: [-0.024482257664203644, -0.0698036327958107, -0.0394330658018589, ...]

Message: IPW4-88 2 Room Temp
Embedding size: 512
Embedding: [-0.03876537084579468, 0.002980988472700119, -0.03897960111498833, ...]

Message: Room SPA Feedback
Embedding size: 512
Embedding: [0.04019777476787567, -0.0257947389036417, -0.044073011726140976, ...]

Message: Valve Position
Embedding size: 512
Embedding: [-0.042262062430381775, 0.0528494156897068, -0.03955838084220886, ...]

Message: Room Final SPA
Embedding size: 512
Embedding: [0.0306182149797678, 0.03271699324250221, -0.056420404464006424, ...]

Message: Final Room SPA
Embedding size: 512
Embedding: [0.014816020615398884, 0.03497190400

### Get data

In [ ]:
pd.DataFrame(message_embeddings).to_csv("features.csv", index=0)
files.download('features.csv')

### Build Evaluation Graph

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


### Evaluate Sentence Embeddings